## Data Diary

1. step one was to watch the tutorial
https://www.youtube.com/watch?v=QNKxzkNpsko


2. Then i started to scrape. My goal is to access the website, download the xlsx file from it.
Several challenges I face here: (a) I was blocked by the website at first (b) I need to figure out how to locate the xlsx file and dowanload it.

In [ ]:
#After deciding to use beautiful soup and requests to scrape the website. I tried:

my_url = "https://www.trade.gov.tw/Pages/Detail.aspx?nodeID=1375&pid=790878&dl_DateRange=all&txt_SD=&txt_ED=&txt_Keyword=&pageindex=1&history="
raw_html = requests.get(my_url, headers=headers)

#I was blocked by the website so I added headers
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
    "Connection": "keep-alive",
}

#and it worked!

In [ ]:
#after checking the website html i use the a tag that starts with /App_Ashx/File.ashx?FileID= to locate the xlsx download link. and then use urljoin function to join it with the base link to download 

base_url = "https://www.trade.gov.tw"

file_tag = soup_doc.find('a', href=re.compile(r"^/App_Ashx/File.ashx\?FileID="))
file_href = file_tag['href']

full_url = urljoin(base_url, file_href)
print(f"Full file download link: {full_url}")

## Clean data

I need to clean the data downloaded from the International Trade Administration of Taiwan, which presents several challenges:

(a) The header is incorrect
(b) The data is in Mandarin
(c) The years are not in Common Era format

In [ ]:
#so i make row 2 into my header
df.columns = df.iloc[1]

#then i drop the rows that are not relevant
df = df.drop([0, 1, 2, df.index[-3], df.index[-2], df.index[-1]]).reset_index(drop=True)

#and the ones are NaN (those are the empty rows saved for future months)
df = df[~df.iloc[:, 1:].isna().all(axis=1)]

In [ ]:
#i translated the column names into English
df.columns = ['Month/Year', 'Total', 'Export', 'Import', 'Balance']

In [ ]:
#I'm setting a function to transform ROC years into Common Era format. First I need to get rid of the "年" (Mandarin word for "year") and turn it into integer and then +1911 (the difference between Common Era format and ROC date format). I'm also replacing the Mandarin months with English names.

def transform_date(value):
    value = str(value)
    if '年' in value:
        year = int(value.replace('年', '').strip())
        return 1911 + year
    elif '月' in value:
        month_map = {
            '1月': 'Jan.', '2月': 'Feb.', '3月': 'Mar.', '4月': 'Apr.',
            '5月': 'May', '6月': 'Jun.', '7月': 'Jul.', '8月': 'Aug.',
            '9月': 'Sep.', '10月': 'Oct.', '11月': 'Nov.', '12月': 'Dec.'
        }
        return month_map.get(value, value)
    return value

df.loc[:, 'Month/Year'] = df['Month/Year'].apply(transform_date)

Then i upload everything to github
set up github action, set the yml to scrape the website once a month
set up the live website https://ashley-yihui-lee.github.io/us-taiwan-trade/
modify the html and added the graph from datawrapper

The only challenge I encountered was that the YAML and GitHub Action didn’t automatically update as shown in the tutorial. After speaking with the TA, I learned that the issue was because the site I scrape only updates once a month, unlike the real-time AQI site in the tutorial. Since there were no changes, nothing updated on GitHub.

and side note: I noticed the example website https://laurabejder.com/european_energy_prices/ has a lighting icon, so I modified my HTML to add a Taiwan icon! That’s my favorite part of the project.